In [1]:
def shadow(limit=200):
    def decorator(gen_func):
        def wrapper(*args, **kwargs):
            total = 0
            for item in gen_func(*args, **kwargs):
                try:
                    parts = item.strip().split()
                    if len(parts) != 2:
                        continue
                    action, value = parts
                    value = int(value)
                except ValueError:
                    continue
                print(f"Transaction: {action} {value}")
                total += value
                if total > limit:
                    print("Shadow limit exceeded. Active schema")
                yield item
            return total
        return wrapper
    return decorator


@shadow(limit=200)
def transaction_stream():
    transactions = [
        "payment 120",
        "refund 50",
        "transfer 300",
        "bad_data xyz",   
        "payment 100"
    ]
    for t in transactions:
        yield t
if __name__ == "__main__":
    gen = transaction_stream()
    try:
        while True:
            next(gen)
    except StopIteration as e:
        print(f"Final sum: {e.value}")

Transaction: payment 120
Transaction: refund 50
Transaction: transfer 300
Shadow limit exceeded. Active schema
Transaction: payment 100
Shadow limit exceeded. Active schema
Final sum: 570
